In [79]:
# from open ai cookbook:
# https://cookbook.openai.com/examples/assistants_api_overview_python

from openai import OpenAI
from dotenv import load_dotenv
import os
import json

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

file = client.files.create(
    file=open("memgpt-paper.pdf", "rb"),
    purpose = "assistants"
)
print(file)

FileObject(id='file-m2sr1SpetNbbpVqWsyoeDd0h', bytes=663708, created_at=1719960616, filename='memgpt-paper.pdf', object='file', purpose='assistants', status='processed', status_details=None)


In [94]:
#step 1 - create an assistant
assistant = client.beta.assistants.create(
    name = "Machine learning researcher",
    instructions = "You are a machine learning researcher. Answer questions using the research paper.",
    tools = [{"type": "file_search"}],
    model = "gpt-4o"
)

In [95]:
#step 2 - create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_RIwFym7lee7wDXURuWfWj3yi', created_at=1719960929, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [96]:
#step 3 - add a message to the thread
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "summarize the research paper"
)
print(message)

Message(id='msg_Jnjz75nNV8H9zQEv23R2C9pC', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='summarize the research paper'), type='text')], created_at=1719960931, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_RIwFym7lee7wDXURuWfWj3yi')


In [97]:
#step 4 - run the assistant
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)
print(run)

Run(id='run_ezKNZdixP3n1WdCGutsQkid2', assistant_id='asst_aQy5Jz3jlrGjlywioT52muEI', cancelled_at=None, completed_at=None, created_at=1719960934, expires_at=1719961534, failed_at=None, incomplete_details=None, instructions='You are a machine learning researcher. Answer questions on the research paper.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_RIwFym7lee7wDXURuWfWj3yi', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=None)], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [101]:
#step 5 - display the assistant's response
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id, 
    run_id = run.id
)
print(run)

Run(id='run_ezKNZdixP3n1WdCGutsQkid2', assistant_id='asst_aQy5Jz3jlrGjlywioT52muEI', cancelled_at=None, completed_at=1719960936, created_at=1719960934, expires_at=None, failed_at=None, incomplete_details=None, instructions='You are a machine learning researcher. Answer questions on the research paper.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1719960934, status='completed', thread_id='thread_RIwFym7lee7wDXURuWfWj3yi', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=None)], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=44, prompt_tokens=1225, total_tokens=1269), temperature=1.0, top_p=1.0, tool_resources={})


In [102]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)
print(messages)

SyncCursorPage[Message](data=[Message(id='msg_xHlB5J0IMMImHBc3xv4pp9eO', assistant_id='asst_aQy5Jz3jlrGjlywioT52muEI', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="I couldn't find a research paper uploaded for summarization. Could you please upload the research paper so I can provide a summary for you?"), type='text')], created_at=1719960936, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_ezKNZdixP3n1WdCGutsQkid2', status=None, thread_id='thread_RIwFym7lee7wDXURuWfWj3yi'), Message(id='msg_Jnjz75nNV8H9zQEv23R2C9pC', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='summarize the research paper'), type='text')], created_at=1719960931, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_RIwFym7lee7wDXURuWfWj3yi')], object='list

In [104]:
for message in reversed(messages.data): 
    print(message.role + ": " + message.content[0].text.value)

user: summarize the research paper
assistant: I couldn't find a research paper uploaded for summarization. Could you please upload the research paper so I can provide a summary for you?
